[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fanzheng10/HiDeF/blob/master/analysis/simple_app.ipynb)


# Install dependencies

In [ ]:
!pip install dash  &> /dev/null
!pip install dash-cytoscape  &> /dev/null
!pip install jupyter-dash  &> /dev/null
!apt-get -qq install -y graphviz && pip install pydot  &> /dev/null

In [ ]:
!pip install hidef &> /dev/null

In [ ]:
import os, sys
import pandas as pd
import json
import networkx as nx

In [ ]:
import dash
import dash_cytoscape as cyto
from dash import html
from jupyter_dash import JupyterDash
from dash import dcc
from dash.dependencies import Input, Output, State


In [ ]:
from hidef import *
from hidef.dash_utils import *

# A simple model

In [ ]:
P = ['11111111',
  '11111100',
  '00001111',
  '11100000',
  '00110000',
  '00001100',
  '00000011']

wv = weaver.Weaver()
H = wv.weave(P, cutoff=1.0)
Hv = wv.show()

Use dash_cytoscape to create a more dynamic network visualization

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape',
        elements=hier_digraph_to_dash(H, Hv[1]),
        layout={'name': 'breadthfirst',
                'roots': '[id = "0-0"]'}
    )
])
app.run_server(mode='inline')

In [ ]:
# TODO: this should be a function
app = JupyterDash(__name__)
app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape',
        elements=hier_digraph_to_dash(wv, Hv[1], True),
        layout={'name': 'breadthfirst',
                'roots': '[id = "0-0"]'}
    )
])
app.run_server(mode='inline')

Add a panel to show node data

In [ ]:
app = JupyterDash(__name__)

styles = {
    'json-output': {
        'overflow-y': 'scroll',
        'height': 'calc(100% - 25px)',
        'border': 'thin lightgrey solid'
    },
    'tab': {'height': 'calc(98vh - 115px)'}
}

app.layout = html.Div([
  html.Div([
      cyto.Cytoscape(
          id='cytoscape',
          elements=hier_digraph_to_dash(wv, Hv[1], True, True),
          layout={'name': 'breadthfirst',
                  'roots': '[id = "0-0"]'}
      )
  ]),
  html.Div(className='four columns', children=[
      dcc.Tab(label='Mouseover Data', children=[
          html.Div(style=styles['tab'], children=[
              html.P('Node Data JSON:'),
              html.Pre(
                  id='mouseover-node-data-json-output',
                  style=styles['json-output']
              )
          ])
      ]),
  ])
],style={'display': 'flex', 'flex-direction': 'row'})

@app.callback(Output('mouseover-node-data-json-output', 'children'),
              [Input('cytoscape', 'tapNodeData')])
def displayMouseoverNodeData(data):
    return json.dumps(data, indent=2)

app.run_server(mode='inline')
